<a href="https://colab.research.google.com/github/chakshu-dhannawat/ML-Experimentation/blob/main/BLOOM_Language_Model_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Reference:
#https://huggingface.co/blog/bloom

In [ ]:
#In this file, I have experimented with the language model BLOOM, and made inferences through it

In [1]:
# requirements:
! pip install huggingface_hub
! git config --global credential.helper store

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 101 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from huggingface_hub import notebook_login
from huggingface_hub import HfFolder


#enter your API key, you can make one for free on HF
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [3]:
from huggingface_hub import InferenceApi

inference = InferenceApi("bigscience/bloom",token=HfFolder.get_token())

In [17]:
help(inference)

Help on InferenceApi in module huggingface_hub.inference_api object:

class InferenceApi(builtins.object)
 |  InferenceApi(repo_id: str, task: Union[str, NoneType] = None, token: Union[str, NoneType] = None, gpu: Union[bool, NoneType] = False)
 |  
 |  Client to configure requests and make calls to the HuggingFace Inference API.
 |  
 |  Example:
 |  
 |  ```python
 |  >>> from huggingface_hub.inference_api import InferenceApi
 |  
 |  >>> # Mask-fill example
 |  >>> inference = InferenceApi("bert-base-uncased")
 |  >>> inference(inputs="The goal of life is [MASK].")
 |  [{'sequence': 'the goal of life is life.', 'score': 0.10933292657136917, 'token': 2166, 'token_str': 'life'}]
 |  
 |  >>> # Question Answering example
 |  >>> inference = InferenceApi("deepset/roberta-base-squad2")
 |  >>> inputs = {
 |  ...     "question": "What's my name?",
 |  ...     "context": "My name is Clara and I live in Berkeley.",
 |  ... }
 |  >>> inference(inputs)
 |  {'score': 0.9326569437980652, 'start'

In [32]:
import time

def infer(prompt,
          max_length = 128,
          top_k = 0,
          num_beams = 0,
          no_repeat_ngram_size = 2,
          top_p = 0.9,
          seed=42,
          temperature=0.7,
          greedy_decoding = False,
          return_full_text = False):
    

    top_k = None if top_k == 0 else top_k
    do_sample = False if num_beams > 0 else not greedy_decoding
    num_beams = None if (greedy_decoding or num_beams == 0) else num_beams
    no_repeat_ngram_size = None if num_beams is None else no_repeat_ngram_size
    top_p = None if num_beams else top_p
    early_stopping = None if num_beams is None else num_beams > 0

    params = {
        "max_new_tokens": max_length,
        "top_k": top_k,
        "top_p": top_p,
        "temperature": temperature,
        "do_sample": do_sample,
        "seed": seed,
        "early_stopping":early_stopping,
        "no_repeat_ngram_size":no_repeat_ngram_size,
        "num_beams":num_beams,
        "return_full_text":return_full_text
    }
    
    s = time.time()
    response = inference(prompt, params=params)
    #print(response)
    proc_time = time.time()-s
    #print(f"Processing time was {proc_time} seconds")
    return response

##Basic Prompts

In [7]:
prompt = "The thing that makes large language models interesting is"
resp = infer(prompt)

resp

[{'generated_text': 'The thing that makes large language models interesting is that they are huge. They have so many parameters that the cost of training is used to check whether a given property is defined. The property can be given as an operator in the factor graph of the original problem. This leads to the following equivalence.\nTheorem 2. The local, el tradicional, el de la cuina'}]

In [11]:
prompt = " #Use OpenCV in Python"
resp = infer(prompt)

resp

[{'generated_text': ' #Use OpenCV in Python for face recognition (a bit tricky) #OpenCV and Python (a bit tricky) #Face recognition using Python and OpenCV (a bit tricky) ########\nthe house, and brought in the news that Mr. Tuke is a character that this is a new kind of paper, but it is not'}]

##Advanced Prompting (Predifined format of input, forcing the language model to think in a particular way)

In [27]:
prompt = """ **ChatBot**
Person: What should I do today? What are my daily tasks?
Bot:"""
resp = infer(prompt)

print(resp[0]['generated_text'])

 **ChatBot**
Person: What should I do today? What are my daily tasks?
Bot: Hi, my name is *BotName*, I am your personal assistant. You can ask me anything and I will help you. What should I do today?
Person: I need to buy some food for my family.
Bot: I see, I will help you with that. You can ask me anything about food


In [24]:
inp =""" **Review: **
I was apprehensive about ordering a recliner online, so I did extensive research about it, and found that the best deal was available on amazon and the recliner was also comfortable.


**Questions: **
1. What product is this a review of?
2. Is this review overall positive or negative?
3. What are the main points?

**Answers: **
1. """

resp = infer(inp)

print(resp[0]['generated_text'])

 **Review: **
I was apprehensive about ordering a recliner online, so I did extensive research about it, and found that the best deal was available on amazon and the recliner was also comfortable.


**Questions: **
1. What product is this a review of?
2. Is this review overall positive or negative?
3. What are the main points?

**Answers: **
1.  This is a review of a recliner chair.
2. The review is overall positive. The reviewer says the chair is comfortable, and the price is fair.
3. The review says that the chair is comfortable, and that the price is fair.


**Notes: **
* This is a review question.



In [28]:
inp =""" **Python Code**
Write code to add two numbers in python. """

resp = infer(inp)

print(resp[0]['generated_text'])

 **Python Code**
Write code to add two numbers in python. 

    """
def main():
    x = int(input("Enter a number:"))
    y = int(input("Enter another number:"))
    z = x + y
    print("The sum of ",x," and ",y," is ",z)

if __name__ == "__main__":
    main


In [39]:
inp =""" **Sample Essay**
*Best things in life are free*
Nature has a wide variety of things to offer. """

resp = infer(inp, max_length = 128)

print(resp[0]['generated_text'])

 **Sample Essay**
*Best things in life are free*
Nature has a wide variety of things to offer.  It is a gift from God. It is the best gift we have ever received. It is a gift that will never run out. Nature has its own beauty and wonder that is hard to describe. Nature is not only beautiful but it is also very important to our lives. I believe that nature is one of the best things in life. I think that nature is the best thing in life because it helps us to have a better life. Nature gives us fresh air, clean water and a beautiful environment to live in. I think that nature is the best thing in life because it makes us happy and healthy. It makes us healthy because it


In [34]:
inp =""" **Python Code**
Code to find nth fibonacci number in python."""

resp = infer(inp)

print(resp[0]['generated_text'])

 **Python Code**
Code to find nth fibonacci number in python. The code has been tested and is working fine. I have used a recursive function to find nth fibonacci number. The idea of the code is to keep on adding the previous two numbers in the sequence till the nth number is obtained. The code is as follows:
def Fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return Fibonacci(n-2) + Fibonacci(n-1)

print(Fibonacci(10))Introduction
The primary role of the health care system is to provide high quality health care to patients. A key component in providing high quality


Below prompt is from a text of Allan Johnson - "The Forests and the Trees". Lets see how well the model understands the underlying information.

In [47]:
inp =""" **Sample Essay**
*Sociology*
We should understand how we are participating in the social syatem and not have an individualistic perspective."""

resp = infer(inp, max_length = 128)

print(resp[0]['generated_text'])

 **Sample Essay**
*Sociology*
We should understand how we are participating in the social syatem and not have an individualistic perspective. Individualistic perspectives have many negative consequences such as social isolation, lack of empathy, and loss of the community. Society has many problems because of individualism. We need to have a perspective that takes into account the community. Sociology helps us to understand the problems of society and the community. It helps us understand that we are not alone in the world. We need to understand that the St. Petersburg and Moscow metro stations are the largest in the world, but did you know that the Moscow metro is the deepest and longest in the world? In total, there are 12 lines and more than 200 stations, but the number of stations


My Take: There is still a long way to go, but large language models are giving promising results :)